# Export raw tick data using dedicated service - F#

### Overview
This sample demonstrates how to request from a dedicated raw data service **on-demand** tick by tick data `for a given instrument or a given watchlist`. 

### Inputs/outputs
Tick data extraction sample requires instrument's identifier, date time intervals as per inputs and exports tick by tick data file. 

### Services used
This sample uses *gRPC requests* in order to retrieve tick by tick dataset from the hosted service. The queried endpoint in this script are:
* *TickRawService*: to directly retrieve raw dataset from the server.

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Tick.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    
***

# Run tick by tick dataset extraction sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 0.*-pre*"
#r "nuget: TaskBuilder.fs"

In [ ]:
open Systemathics.Apis.Helpers
open Systemathics.Apis.Type.Shared.V1
open Systemathics.Apis.Services.Tick.V1
open Google.Protobuf.WellKnownTypes
open Google.Type
open Grpc.Net.Client
open Grpc.Core
open System.IO
open System.Globalization

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
let headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
let channel = ChannelHelpers.GetChannel();

### Step 3: Create and process request
To request *raw tick data* service, we need to specify instrument identifier and a period of time. The following code snippets ensure required objects are created and proceed to request instantiation.

#### 3.1 Instrument selection

In [ ]:
let ticker = "AAPL"
let exchange = "BATS"

#### 3.2 Time period delimitation

In [ ]:
let start = new DateTime(2021, 03, 05)
let end' = new DateTime(2021, 03, 05)
let dateIntervals = 
  new DateInterval(
    StartDate = new Date ( Year = start.Year, Month = start.Month, Day = start.Day ),
    EndDate = new Date ( Year = end'.Year, Month = end'.Month, Day = end'.Day )
  )

// Build the market data request time interval (we are using Google date time format : we have to cast the dates)
// UTC time zone
let timeInterval = 
  new TimeInterval(
     StartTime = new TimeOfDay ( Hours = 14, Minutes = 00, Seconds = 00 ),
     EndTime = new TimeOfDay ( Hours = 14, Minutes = 10, Seconds = 00 ) 
  )

#### 3.3 Request creation
The following code snippet creates *gRPC client*, process request and returns the request reply:

In [ ]:
// Generate constraints based on the previous space selection:
let constraints = new Constraints() 
constraints.DateIntervals.Add(dateIntervals)
constraints.TimeIntervals.Add(timeInterval)

The following code snippets create the market data request and instantiate the service:

In [ ]:
// Generate raw tick data request
let request = 
    new TickRawRequest(
        Identifier = new Identifier ( Exchange = exchange, Ticker = ticker), 
        Constraints = constraints,
        Level = Level.TradesAndBook
    )

// Instantiate the raw tick data service
let service = new TickRawService.TickRawServiceClient(channel)

### Step 4: Retrieve tick by tick data

Tick by tick data is sent through a *stream*, the following code snippet exports tick by tick dataset in a file in the same folder as this sample:

In [ ]:
open FSharp.Control.Tasks.V2
open System.Collections.Generic

let iterAsync (t: IAsyncEnumerable<'T>) (f : 'T -> unit) =
    let task = 
        task {
            let e = t.GetAsyncEnumerator()
            let mutable go = true
            try
                let! step = e.MoveNextAsync()
                go <- step
                while go do 
                    f e.Current
                    let! step = e.MoveNextAsync()
                    go <- step
            finally
                e.DisposeAsync().AsTask().Wait()
        }
    task.Wait()

let write (fs : StreamWriter) (response : TickRawResponse) =
    let subString2 (d : DateTime) (str : string) =
        $"{d:O} {str.Substring(2)}"

    let write' (i : TickRawItem) =
        i.Payload.ToByteArray()
        |> Encoding.ASCII.GetString
        |> subString2 (i.TimeStamp.ToDateTime())
        |> fs.WriteLine

    response.RawItems
    |> Seq.iter write'

let call = service.TickRaw(request, headers)
let enumerator = call.ResponseStream.ReadAllAsync()
using (new StreamWriter($"{ticker}_raw_ticks.txt")) (fun sw -> iterAsync enumerator (write sw))
